In [1]:
## Import necessary libraries
import pandas as pd

from PortfolioTracker.globalpath import GlobalPath

In [2]:
# Instantiate GlobalPath
global_path = GlobalPath("PortfolioTracker")
# GLOBAL PATH

holdings_gold_file_path = global_path.joinpath(
    "DATA/GOLD/Holdings/Holdings_data.csv"
)

symbol_silver_file_path = global_path.joinpath(
    "DATA/SILVER/Symbol/Symbol_data.csv"
)

stock_holding_records_file_path = global_path.joinpath(
    "DATA/GOLD/Holdings/HoldingsRecords_data.csv"
)

In [3]:
# Load holdings data
df_holdings = pd.read_csv(holdings_gold_file_path)
df_holdings["date"] = pd.to_datetime(df_holdings["date"])
print(f"Loaded holdings data from: {holdings_gold_file_path}")

Loaded holdings data from: ..\..\DATA\GOLD\Holdings\Holdings_data.csv


In [4]:
# Group by 'segment', 'exchange', and 'symbol', and calculate the min and max dates for each group
df_stock_holding_records = (
    df_holdings.groupby(["segment", "exchange", "symbol"])
    .agg(min_date=("date", "min"), max_date=("date", "max"))
    .reset_index()
)

In [5]:
# Replace scrip code with company name
df_symbol = pd.read_csv(symbol_silver_file_path)

# Merge df_TradeHistory with df_Symbol on the matching columns
df_stock_holding_records = df_stock_holding_records.merge(
    df_symbol[["symbol", "scrip_name", "isin"]],
    left_on="symbol",
    right_on="symbol",
    how="left",
)

In [6]:
# Select and reorder columns for the final output format
df_stock_holding_records = df_stock_holding_records[
    [
        "segment",
        "exchange",
        "symbol",
        "scrip_name",
        "isin",
        "min_date",
        "max_date",
    ]
]

# Sort the DataFrame by 'date' and reset the index for organized output
df_stock_holding_records = df_stock_holding_records.sort_values(
    by=["segment", "exchange", "symbol"]
).reset_index(drop=True)

# Save the resulting DataFrame to a new CSV file
df_stock_holding_records.to_csv(stock_holding_records_file_path, index=False)
print(
    f"Stock holding records CSV file created at: {stock_holding_records_file_path}"
)

# Display the DataFrame debugrmation for verification
df_stock_holding_records.info()

Stock holding records CSV file created at: ..\..\DATA\GOLD\Holdings\HoldingsRecords_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   segment     19 non-null     object        
 1   exchange    19 non-null     object        
 2   symbol      19 non-null     object        
 3   scrip_name  19 non-null     object        
 4   isin     19 non-null     object        
 5   min_date    19 non-null     datetime64[ns]
 6   max_date    19 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(5)
memory usage: 1.2+ KB
